# Regridding in CDAT

Whenever you're dealing with CMIP data you will likely encounter the issue of "regridding", i.e going from one model's grid to another

CDAT comes with 3 different regridders: *regrid2*, *esmf* and *libcf*

## regrid2

Regrid2 is CDAT's original regridder, it an area-based regridder that works only on rectilinear grids, for more info see [this PCMDI report](https://pcmdi.llnl.gov/report/pdf/34.pdf?id=76)

## scripp

Scripp regridder is possible provided that you have the scripp weight file.

## libcf

libcf is the second regridder ported to CDAT, it is not maintained anymore as in no new feature will be added. It was developped by unicar, for more info see [this page](https://www.unidata.ucar.edu/software/libcf/html/group__gs__regrid__grp.html)

## esmf

ESMF regridder is currently CDAT's default regridder. It was integrated with CDAT to enable easy use of this powerful tool.

## Default regridder

Regridding in CDAT is done via the `cdms2` module, the default regridder *tool* is `esmf`, the default regrid *method* is `linear`


# Examples

In [1]:
import cdms2
miroc_ps_file = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/r1i1p1f1/Amon/ps/gn/v20181212/ps_Amon_MIROC6_historical_r1i1p1f1_gn_195001-201412.nc")
ipsl_ps_file = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ps/gr/v20180803/ps_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
ipsl = ipsl_ps_file("ps", time=("2000", "2001", "con"))
miroc = miroc_ps_file["ps"]  # No need to read data in at all

print("Original shape:", ipsl.shape)
ipsl_ps_on_miroc_grid_esmf_linear = ipsl.regrid(miroc.getGrid())
print("New shape:", ipsl_ps_on_miroc_grid_esmf_linear.shape)


Original shape: (12, 143, 144)


/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1347: Warning: 
avariable.regrid: We chose regridTool = esmf for you among the following choices:
   Tools ->    'regrid2' (old behavior)
               'esmf' (conserve, patch, linear) or
               'libcf' (linear)
  warnings.warn(message, Warning)
/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1354: Warning: 
avariable.regrid: We chose regridMethod = linear for you among the following choices:
    'conserve' or 'linear' or 'patch'
  warnings.warn(message, Warning)


New shape: (12, 128, 256)


In [3]:
miroc = miroc(time=("2000", "2001", "con"))
print("Max diff:", (miroc-ipsl_ps_on_miroc_grid_esmf_linear).max())

Max diff: 12857.273


In [7]:
# Let's use all possible regridder
for tool, method in zip(["esmf", "esmf", "esmf", "regrid2", "libcf"], ["conserve", "patch", "linear", "ignored", "ignored"]):
    ipsl_ps_on_miroc_grid = ipsl.regrid(miroc.getGrid(), regridTool=tool, regridMethod=method)
    maxdiff = (miroc - ipsl_ps_on_miroc_grid).max()
    print("Max diff using {}/{} is: {}".format(tool, method, maxdiff))


coordMin =  -90.00, boundMin =  -90.00, coordMax =   90.00, boundMax =   90.00
              
Max diff using esmf/conserve is: 12954.3671875
Max diff using esmf/patch is: 11339.671875
Max diff using esmf/linear is: 12857.2734375
Max diff using regrid2/ignored is: 12964.1953125
Max diff using libcf/ignored is: 12857.25


Please proceed to [cdms manual regrid notebook](01_cdms_chapter_4_regrid.ipynb)